# Problema 3 - Análisis Datos Youtube - Archivo .py

1. Descargar un archivo .zip mediante código del siguiente url (https://netsg.cs.sfu.ca/youtubedata/) (recomiendo descargar el archivo 0333.zip que es menos pesado)
2. Descomprimir los datos en una carpeta que genere y leer mediante pandas alguno de los archivos en esta. (observar que no es necesario en un primer momento leer los datos con un nombre de columna especifico)

    - Los nombres de columna pueden ser puestos posteriormente
    - El separador de columna es <code>\t</code>
    - Se colocan los nombres de columnas y descripción asociada para su intermetación. Ejemplo columna1 sera VideoID ... 
    

3. Procesar los datos según: 
    - Nos quedaremos con las columnas: VideoID, edad, catgoria, views, rate.
    - Realizar un filtrado básico a los datos. Ejemplo solo seleccionar cierto grupo de categorias

4. Procesamiento en Mongo Db
    - Exportar los datos a mongo DB 
    - Compartir link donde encontrar los datos 




In [ ]:
import requests, zipfile, io

url = "https://netsg.cs.sfu.ca/youtubedata/0333.zip"
r = requests.get(url)
r.raise_for_status()
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall("datos_youtube_0333")

import pandas as pd
import glob
from zipfile import ZipFile

# Encuentra todos los .txt dentro de esa carpeta:
filepaths = glob.glob("datos_youtube_0333/*.txt")

dfs = []
for fp in filepaths:
    # leer cada archivo como TSV sin encabezado
    df = pd.read_csv(fp, sep="\t", header=None)
    dfs.append(df)

df_all = pd.concat(dfs, ignore_index=True)


In [ ]:
df = df_all[["VideoID", "edad", "categoria", "views", "rate"]]

# por ejemplo: quedarse solo con categorías determinadas:
categorias_validas = ["Music", "Sports", "News"]
df_filtered = df[df["categoria"].isin(categorias_validas)]


In [ ]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")
db = client["youtube_db"]
collection = db["videos"]

# Si ya existen datos, puedes limpiar:
collection.drop()

# Convertir DataFrame a diccionarios y guardar:
records = df_filtered.to_dict(orient="records")
collection.insert_many(records)

print(f"Guardados {len(records)} documentos en MongoDB")


| Nombre de la Columna | Descripción                                                                                                 |
|----------------------|-------------------------------------------------------------------------------------------------------------|
| `video ID`           | Una cadena de 11 dígitos, la cual es única                                                                |
| `uploader`           | Una cadena con el nombre de usuario del cargador del video                                                  |
| `age`                | Un número entero que representa los días transcurridos desde la fecha en que se subió el video hasta el 15 de febrero de 2007 (fecha de creación de YouTube) |
| `category`           | Una cadena que indica la categoría del video elegida por el cargador                                       |
| `length`             | Un número entero que representa la duración del video en minutos                                            |
| `views`              | Un número entero que representa el número de visualizaciones del video                                      |
| `rate`               | Un número flotante que indica la calificación del video                                                      |
| `ratings`            | Un número entero que representa el número de calificaciones recibidas por el video                          |
| `comments`           | Un número entero que indica el número de comentarios en el video                                            |
| `related IDs`        | Hasta 20 cadenas de texto con los IDs de videos relacionados                                                |
